# Variational Autoencoder

Before starting to code, let's review the theory behind Variational Autoencoders (VAEs). 
The goal of VAEs is to learn the underlying probability distribution of the data it is trained on - i.e., to model the latent space of the data. The latent space should capture the essence of the data in a more compact way. But with what kind of (prior) distribution over the latent space, $p(z)$, should we start?

<br />

### Choice of the latent distribution prior $p(z)$
As we want it to be continuous for later sampling, the simplest possible prior is a standard normal distribution: $p(z)=\mathcal{N}(z|0,\mathcal{I})$

However, it is unlikely that this prior captures the characteristics of the observed data (e.g., the images that we plan to put into the VAE). How can we tailor the latent distribution to the specifics of the given data - or differently, how can we learn $p(z|x)$? 

<br />

### Variational Inference (deriving the ELBO)
To answer this, let's make a quick excursion to Variational Inference (VI). VI is a machine learning method that aims to approximate a probability distribution through optimization. This opens the question how we can optimize a distribution to capture the characteristics of the input data. To answer this, let's start with Bayes Formula:

$p(z|x) = \frac{p(x|z)p(z)}{p(x)}$

The problem here is that $p(x)$ is intractable as it involves the integral over all possible values of the latent variable, z, which is usually high-dimensional: $p(x) = \int p(x, z) \, dz$

This means we cannot just get the posterior distribution $p(z|x)$. But, we can approximate it with an easier-to-work with surrogate distribution that we will call $q(z)$. As a metric to see how close this surrogate matches $p(z|x)$, we may use the KL Divergence. The KL Divergence is a measure of how similar (or distant) two distributions are. 

$\text{KL}(q(z) || p(z|x)) = \int q(z) \log\left(\frac{q(z)}{p(z|x)}\right) \, dz$

Since we don't have $p(z|x)$, let's rewrite it:

$p(z|x)=\frac{p(x|z)p(z)}{p(x)}=\frac{p(x,z)}{p(x)}$

So, we can replace $p(z|x)$ in the upper equation:

$\text{KL}(q(z) || \frac{p(x,z)}{p(x)}) = \int q(z) \log\left(\frac{q(z)p(x)}{p(x,z)}\right) \, dz$

Additionally, we can change the numerator and denominator in the log expression, by putting a minus in front of the equation (which equalizes $log()^{-1}$):

$\text{KL}(q(z) || \frac{p(x,z)}{p(x)}) =-\int q(z) \log\left(\frac{p(x,z)}{q(z)p(x)}\right) \, dz$

Then, let's put $p(x)$ (which we don't have) into a separate log:

$\text{KL}(q(z) || \frac{p(x,z)}{p(x)}) =-\int q(z) \log\left(\frac{p(x,z)}{q(z)}\right) \, dz + \int q(z) \log{p(x)}\, dz$

The expectation over something that does not contain z equals what the expectation was taken over, i.e.: $\int q(z) \log{p(x)}\, dz = \mathbb{E}_{q(z)}[\log p(x|z)]= \log{p(x)}$

So, we have:

$\text{KL}(q(z) || \frac{p(x,z)}{p(x)}) =-\int q(z) \log\left(\frac{p(x,z)}{q(z)}\right) \, dz + \log{p(x)}$

It may look like we just shifted things around, but notice a few things:

1. A probability, like $p(x)$, ranges between 0 and 1. The log of something between 0 and 1 will always be negative or 0. I.e.: $\log{p(x)}\leq0$
2. We don't have $p(x)$, but we know that it is fixed because we observe the dataset and  the probability does not change depending on what surrogate posterior we choose.
3. The KL Divergence is always positive or 0: $\text{KL}(q(z) || \frac{p(x,z)}{p(x)})\geq0$

Therefore, we can deduce (important!): $\int q(z) \log\left(\frac{p(x,z)}{q(z)}\right) \, dz$ has to be more negative than $\log{p(x)}$, otherwise the KL divergence cannot be positive. That means, $\int q(z) \log\left(\frac{p(x,z)}{q(z)}\right) \, dz$ bounds $\log{p(x)}$. Since $\log{p(x)}$ is called the "evidence", we call $\mathcal{L} = \int q(z) \log\left(\frac{p(x,z)}{q(z)}\right) \, dz$, frequently expressed as $\mathcal{L} = \mathbb{E}_{z \sim q(z)}[\log\left (\frac{p(x,z)}{q(z)}\right)]$, the **Evidence Lower Bound (ELBO)**.

Thus, we have something that is computable and that we can optimize to be close to the posterior we always wanted! We need to find the parameters and the distribution $q(z)$ that maximize the ELBO.

<br />

### Learning the parameters of $q(z)$

We have the ELBO now, which enables us to optimize the parameters of $q(z)$. Let's call the parameters $\phi$. Thus, for one input sample, $x^{(i)}$, we could learn the optimal $\phi$.

$\phi^{(i)}_{optimal}=\underset{\phi^{(i)}}{\mathrm{argmax}}\:\mathcal{L}_{i}(\phi^{(i)})$

However, for many samples we would have to learn a separate parameters and how would we obtain the latent features for a new sample? Instead, we may train a neural network, $f_{\lambda}$, mapping every sample in our dataset to the optimal parameters $\phi^{(i)}_{optimal}$. In fact, this neural network is going to be the **Encoder** of our VAE!

$\underset{\lambda}{\mathrm{argmax}}\:\frac {1}{N}\sum_{i=1}^{N}\mathcal{L}_{i}(f_{\lambda}(x^{(i)}))$

Long story short - this is what we want to optimize in the Encoder part. Let's look at the Decoder, now.

<br />

### Choosing the output distribution $p(x|z)$
For every z, we want to have a probability distribution over x that maximizes the likelihood of getting back our input.
Let's assume a a gaussian distribution again: $p(x|z)=\mathcal{N}(x|\mu,\mathcal{I})$, parameterized by $\theta$.

Similar to $q(z)$, we want to learn the parameters with a neural network, $g_{\gamma}$, which is going to be our **Decoder**. Thus, $\mu = g_{\gamma}(z)$. 
While $p(x|z)$ is "just a gaussian", f models the mean of this gaussian and can introducie complex transformations on the initial distribution.

Let's write out the final loss which we will try to optimize with regards to the parameters of the Encoder and Decoder.

<br />

### VAE Loss Function
This was the ELBO equation we derived: $\mathcal{L} = \mathbb{E}_{z \sim q(z)}[\log\left (\frac{p(x,z)}{q(z)}\right)]$

To get an even better intuition what we are doing, let's rewrite it a little for the last time:

$\mathcal{L}(\phi, \theta) = \mathbb{E}_{z \sim q(z)}[\log (p_{\theta}(x|z)) + \log(p(z)) - \log(q_{\phi}(z))] \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:= \mathbb{E}_{z \sim q(z)}[\log (p_{\theta}(x|z))] - KL(q_{\phi}(z) || p(z))$

From this expression, think about what maximizing $\mathcal{L}(\phi, \theta)$ means:

1. Maximizing the $\mathcal{L}(\phi, \theta)$ means we maximize the likelihood of seeing our x, given z by this $\mathbb{E}_{z \sim q(z)}[\log (p_{\theta}(x|z))]$
2. Maximizing the $\mathcal{L}(\phi, \theta)$ also means that we try to minimize $KL(q_{\phi}(z) || p(z))$ which ensures that $q(z)$ is close to $p(z)$. I.e., the distribution $q(z)$ should not differ too much from the prior we chose.

<br />

### Optimizing the ELBO
1. Compute $\phi=f_{\lambda}(x)$
2. Compute a Monte Carlo estimate of the ELBO
- Draw $z'\sim q_{\phi}(z)$ with reparameterization*
- Compute $\mu = g_{\gamma}(z)$
- ELBO: $\mathcal{L}(\phi, \theta) \approx \log (p_{\theta}(x|z')) - KL(q_{\phi}(z) || p(z))$
3. Backpropagation:
- Compute $\nabla_{\lambda}\mathcal{L}(\phi, \theta)$
- Compute $\nabla_{\gamma}\mathcal{L}(\phi, \theta)$
4. Update $\lambda$ and $\gamma$ (weights of the Encoder and Decoder network) using gradient ascent

Essentially, we train to find the optimal parameters of the latent distribution, $q_{\phi}(z)$ with the Encoder and train to find the optimal parameters of $p_{\theta}(x|z)$ with the Decoder.

<br />

##### *reparameterization
To do backpropagation through the sampling step, we split it into a deterministic ($\mu$ and $\sigma$) and a random part ($\epsilon$) for which no backpropagation is done.

$z'=\sigma*\epsilon+\mu$ 

where $\epsilon \sim \mathcal{N}(\epsilon|0,\mathcal{I})$